In [27]:
#Note: I'm considering and infinite deck of cards

#Import needed libraries
import random

#Define the values for each card of the deck if A = 11
def deck_values():
    deck_values = {}
    hearts = ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K']
    for card in hearts:
        if card in ['J','Q','K']:
            deck_values[card] = 10
        elif card == 'A':
            deck_values[card] = 11
        else:
            deck_values[card] = int(card)
    return(deck_values)    

#Define the values for each card of the deck if A = 1
def deck_values_2():
    deck_values = {}
    hearts = ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K']
    for card in hearts:
        if card in ['J','Q','K']:
            deck_values[card] = 10
        elif card == 'A':
            deck_values[card] = 1
        else:
            deck_values[card] = int(card)
    return(deck_values) 
        
# The player has a total amount of money (pot), defined at the begining of the game. The minimum bet is 5, thus the minimum pot is also 5.
def money_on_the_table():
    pot = 0
    while pot < 5:
        pot = int(input("How much money do you want to bet on this table. The minimum bet is $5\n"))
    return(pot)

#The player places the bet at the beginning of each round
def bet_placing():
    bet = 0
    while (bet < 5) or (bet > 200):
        bet = int(input("How much money do you want to bet on this round?. The maximum bet is $200 and the minimum bet is $5\n"))
        if bet < 5:
            print("The minimum bet is $5")
        elif bet > 200:
            print("The maximum bet is $200")
        else:
            return(bet)
        
#The dealer deals the initial hands for a given deck. Retuns a tuple(?) with 2 lists, 2 items each, the player's hand and the dealer's hand
#WARNING when printing the result, the second card of the dealer's hand should not be shown
def deal_cards(deck):
    player_hand = [random.choice(deck) for x in range(2)]
    dealer_hand = [random.choice(deck) for x in range(2)]
    return player_hand, dealer_hand

#The player chooses what to do
def player_action(values):
    options = ["hit","pass"]
    player_choice = ""
    print("Your hand is",player_hand,"You are now at",hand_sum(player_hand,values))
    while player_choice not in options:
        player_choice = str(input(f"You can {options}, what would you like to do?\n"))
    return player_choice

#We calculate the sum of a hand. Needs a list and a dictionary
def hand_sum(hand,values):
    total = 0
    for card in hand:
        total += values[card]
    return total

#We ask if the player wants to keep playing
def keep_playing():
    if pot >= 5:
        keep = input("Do you want to keep playing Y/N ")
        while keep not in ['Y','N']:
            keep = input("Do you want to keep playing Y/N ")
        return keep == 'Y'
    else:
        print("You don't have enough chips on your pot :(")
        return False
    
#Build the game:
#Ask the player how much money he wants to put in the pot
pot = money_on_the_table()

#We create the deck
deck = ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K']

#We create a loop for each round
give_me_more = True

while give_me_more:
    
    #We ask the player to place a bet for the current round
    print(f'You currently have ${pot} on your pot')
    bet = bet_placing()
    player_busted = False
    dealer_busted = False

    #We deal the cards and display the current status
    print("Dealing...\n...\n...")
    player_hand = deal_cards(deck)[0]
    dealer_hand = deal_cards(deck)[1]
    print("Your hand is:",player_hand[0],player_hand[1],"\nThe dealer's hand is:",dealer_hand[0],"[X]")

    #We give the player the choice to take action
    print("What would you like to do?")

    action = player_action(deck_values())
    player_sum = hand_sum(player_hand,deck_values())
    
    while action == "hit":
        player_hand.append(random.choice(deck))
        player_sum = hand_sum(player_hand,deck_values())
        if player_sum > 21 and 'A' not in player_hand:
            print(player_hand,"\nYou are now at",player_sum,"\nBusted!")
            action = "pass"
            player_busted = True
        elif player_sum > 21 and'A' in player_hand:
            player_sum = hand_sum(player_hand,deck_values_2())
            if player_sum > 21:
                print(player_hand,"\nYou are now at",player_sum,"\nBusted!")
                action = "pass"
                player_busted = True
            else:
                action = player_action(deck_values_2())
        else:
            action = player_action(deck_values())

    #Dealer's play
    dealer_sum = hand_sum(dealer_hand,deck_values())
    #We check if the dealer holds a couple of A's (only hand of 2 cards higher than 21)
    if dealer_sum > 21:
        dealer_sum = hand_sum(dealer_hand,deck_values_2())
    print("The dealer's hand is:",dealer_hand)

    while (dealer_sum < 17) and (player_busted == False):
        dealer_hand.append(random.choice(deck))
        dealer_sum = hand_sum(dealer_hand,deck_values())
        if dealer_sum > 21 and 'A' not in dealer_hand:
            print("The dealer's hand is:",dealer_hand,"\nDealer is busted!")
        elif dealer_sum > 21 and 'A' in dealer_hand:
            dealer_sum = hand_sum(dealer_hand,deck_values_2())
            if dealer_sum > 21:
                print("The dealer's hand is:",dealer_hand,"\nDealer is busted!")
                dealer_busted = True
        else:
            print("The dealer's hand is:",dealer_hand,"\nWith a total score of",dealer_sum)

    #We settle the game and update the pot
    if player_busted:
        pot = pot - bet
        print(f"The dealer wins\n Your remaining pot is ${pot}")
    elif dealer_busted:
        pot = pot + bet
        print(f"You win!\n You have now ${pot} in your pot")
    else:
        if dealer_sum == 21 and player_sum == 21:
            if 'A' in dealer_hand not in player_hand:
                pot = pot - bet
                print(f"The dealer wins\n Your remaining pot is ${pot}")
            elif 'A' in player_hand not in dealer_hand:
                pot = pot + bet
                print(f"You win!\n You have now ${pot} in your pot")
            else:
                print("It's a split")
        else:
            if dealer_sum > player_sum:
                pot = pot - bet
                print(f"The dealer wins\n Your remaining pot is ${pot}")
            elif dealer_sum < player_sum:
                pot = pot + bet
                print(f"You win!\n You have now ${pot} in your pot")
            else:
                print(f"It's a split\n You have now ${pot} in your pot")

    give_me_more = keep_playing()

How much money do you want to bet on this table. The minimum bet is $5
200
You currently have $200 on your pot
How much money do you want to bet on this round?. The maximum bet is $200 and the minimum bet is $5
2
The minimum bet is $5
How much money do you want to bet on this round?. The maximum bet is $200 and the minimum bet is $5
4
The minimum bet is $5
How much money do you want to bet on this round?. The maximum bet is $200 and the minimum bet is $5
5
Dealing...
...
...
Your hand is: A J 
The dealer's hand is: 9 [X]
What would you like to do?
Your hand is ['A', 'J'] You are now at 21
You can ['hit', 'pass'], what would you like to do?
pass
The dealer's hand is: ['9', '4']
The dealer's hand is: ['9', '4', 'Q'] 
Dealer is busted!
The dealer wins
 Your remaining pot is $195
Do you want to keep playing Y/N n
Do you want to keep playing Y/N no
Do you want to keep playing Y/N N
